# 隐患排查与风险可视化系统
## Hazard Risk Dashboard Demo

本Notebook演示了隐患风险监控系统的主要功能和分析结果，包括：
- 数据加载与预处理
- 风险统计分析
- 帕累托分析（关键少数识别）
- 多维度可视化展示
- 3D曲面图分析

---


In [16]:
# 导入必要的库
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 添加src目录到路径
sys.path.insert(0, str(Path('.').absolute() / 'src'))

# 导入项目模块
from data_generator import HazardDataGenerator
from risk_analyzer import RiskAnalyzer
from visualizer import HazardVisualizer

print("[OK] 所有模块导入成功")


[OK] 所有模块导入成功


## 1. 数据加载与概览

加载已生成的隐患数据，或生成新的模拟数据。


In [17]:
# 加载或生成数据
data_path = Path('./data/hazards.csv')

if data_path.exists():
    print("加载现有数据...")
    df = pd.read_csv(data_path)
    df['date'] = pd.to_datetime(df['date'])
else:
    print("生成新的模拟数据...")
    generator = HazardDataGenerator()
    df = generator.generate_all_data()
    generator.save_data(df, Path('./data'))

# 数据概览
print(f"\n[数据集概览]")
print(f"  总记录数: {len(df):,} 条")
print(f"  时间范围: {df['date'].min().strftime('%Y-%m-%d')} 至 {df['date'].max().strftime('%Y-%m-%d')}")
print(f"  区域数量: {df['area_name'].nunique()} 个")
print(f"  隐患类型: {df['hazard_type'].nunique()} 种")
print(f"\n[字段列表]")
print(f"  {list(df.columns)}")


加载现有数据...

[数据集概览]
  总记录数: 4,459 条
  时间范围: 2024-01-01 至 2026-01-02
  区域数量: 9 个
  隐患类型: 35 种

[字段列表]
  ['hazard_id', 'date', 'year', 'month', 'week', 'quarter', 'weekday', 'area_code', 'area_name', 'hazard_category', 'hazard_type', 'description', 'risk_level', 'risk_score', 'status', 'rectify_date', 'rectify_days', 'responsible', 'inspector', 'data_type']


In [18]:
# 查看数据样本
df.head(10)


,hazard_id,date,year,month,week,quarter,weekday,area_code,area_name,hazard_category,hazard_type,description,risk_level,risk_score,status,rectify_date,rectify_days,responsible,inspector,data_type
0,HZ-2024-00001,2024-01-01,2024,1,1,2024Q1,Monday,UTILITIES,公用工程,机械安全,防护罩缺失,公用工程设备防护装置被拆除,Low,1,已整改,2024-01-26,25.0,伍磊,尹娟,historical
1,HZ-2024-00002,2024-01-01,2024,1,1,2024Q1,Monday,NICKEL_COBALT,镍钴车间,行走安全,台阶损坏,镍钴车间楼梯台阶破损,Low,1,已整改,2024-01-13,12.0,何桂英,王莉,historical
2,HZ-2024-00003,2024-01-01,2024,1,1,2024Q1,Monday,FINISHED_GOODS,成品仓库,消防安全,灭火器过期,成品仓库灭火器超过有效期,High,10,已整改,2024-01-02,1.0,陈琴,孟红霞,historical
3,HZ-2024-00004,2024-01-01,2024,1,1,2024Q1,Monday,RAW_MATERIAL,原料仓库,消防安全,烟感故障,原料仓库烟感探测器故障未修复,Low,1,已整改,2024-02-20,50.0,修桂香,王畅,historical
4,HZ-2024-00005,2024-01-01,2024,1,1,2024Q1,Monday,FINISHED_GOODS,成品仓库,个人防护,PPE佩戴不规范,成品仓库作业人员未佩戴安全帽,Low,2,已整改,2024-01-08,7.0,骆玉梅,陈建,historical
5,HZ-2024-00006,2024-01-01,2024,1,1,2024Q1,Monday,RAW_MATERIAL,原料仓库,消防安全,烟感故障,原料仓库烟感探测器故障未修复,Low,1,已整改,2024-01-28,27.0,许明,沈彬,historical
6,HZ-2024-00007,2024-01-02,2024,1,1,2024Q1,Tuesday,UTILITIES,公用工程,个人防护,防护用品缺失,公用工程防护用品配备不足,Medium,5,已整改,2024-01-16,14.0,李建军,李英,historical
7,HZ-2024-00008,2024-01-02,2024,1,1,2024Q1,Tuesday,CONTROL_CENTER,中控室,消防安全,灭火器过期,中控室灭火器超过有效期,Low,1,已整改,2024-01-15,13.0,解玉梅,孙丽,historical
8,HZ-2024-00009,2024-01-02,2024,1,1,2024Q1,Tuesday,ELECTROLYSIS,电解车间,电气安全,配电箱隐患,电解车间配电箱警示标识缺失,Low,2,已整改,2024-01-19,17.0,仲淑珍,陈秀芳,historical
9,HZ-2024-00010,2024-01-02,2024,1,1,2024Q1,Tuesday,UTILITIES,公用工程,消防安全,消防栓问题,公用工程消防栓箱内设备不全,Low,2,已整改,2024-01-31,29.0,杨静,张洁,historical


## 2. 风险统计分析

使用RiskAnalyzer进行全面的风险统计分析。


In [19]:
# 初始化分析器
analyzer = RiskAnalyzer(df)

# 获取汇总统计
stats = analyzer.get_summary_stats()

print("=" * 50)
print("整体风险统计")
print("=" * 50)
print(f"""
┌─────────────────────────────────────────────────┐
│  累计隐患总数    │  {stats['total']:>6,} 条                  │
│  已整改          │  {stats['rectified']:>6,} 条                  │
│  整改中          │  {stats['in_progress']:>6,} 条                  │
│  整改率          │  {stats['rectify_rate']:>6}%                   │
├─────────────────────────────────────────────────┤
│  高风险 (High)   │  {stats['high_risk']:>6,} 条                  │
│  中风险 (Medium) │  {stats['medium_risk']:>6,} 条                  │
│  低风险 (Low)    │  {stats['low_risk']:>6,} 条                  │
├─────────────────────────────────────────────────┤
│  平均风险分      │  {stats['avg_risk_score']:>6}                    │
│  平均整改天数    │  {stats['avg_rectify_days']:>6} 天               │
└─────────────────────────────────────────────────┘
""")


整体风险统计

┌─────────────────────────────────────────────────┐
│  累计隐患总数    │   4,459 条                  │
│  已整改          │   4,349 条                  │
│  整改中          │      74 条                  │
│  整改率          │    97.5%                   │
├─────────────────────────────────────────────────┤
│  高风险 (High)   │     220 条                  │
│  中风险 (Medium) │   1,078 条                  │
│  低风险 (Low)    │   3,161 条                  │
├─────────────────────────────────────────────────┤
│  平均风险分      │     3.2                    │
│  平均整改天数    │    16.9 天               │
└─────────────────────────────────────────────────┘



In [20]:
# 风险等级分布饼图
risk_dist = df['risk_level'].value_counts().reset_index()
risk_dist.columns = ['风险等级', '数量']

# 中文映射
level_names = {'High': '高风险', 'Medium': '中风险', 'Low': '低风险'}
risk_dist['风险等级'] = risk_dist['风险等级'].map(level_names)

colors = {'高风险': '#e74c3c', '中风险': '#f39c12', '低风险': '#2ecc71'}

fig = px.pie(
    risk_dist, 
    values='数量', 
    names='风险等级',
    title='<b>风险等级分布</b>',
    color='风险等级',
    color_discrete_map=colors,
    hole=0.4
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    textfont_size=14
)

fig.update_layout(
    height=400,
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=-0.1, xanchor='center', x=0.5)
)

fig.show()


## 3. 帕累托分析 (80/20法则)

识别导致80%问题的关键20%隐患类型，帮助优先处理高影响因素。


In [21]:
# 帕累托分析
pareto = analyzer.get_pareto_analysis('hazard_type')

print("帕累托分析结果 (隐患类型)")
print("=" * 70)
vital_few = pareto[pareto['is_vital_few'] == True]
print(f"关键少数 (占80%问题): {len(vital_few)} 种隐患类型")
print()
print(pareto.to_string(index=False))


帕累托分析结果 (隐患类型)
关键少数 (占80%问题): 21 种隐患类型

hazard_type  count  percentage  cumulative_percentage  is_vital_few
       临时用电    340         7.6                    7.6          True
      灭火器过期    268         6.0                   13.6          True
      配电箱隐患    253         5.7                   19.3          True
     消防通道堵塞    241         5.4                   24.7          True
       地面湿滑    188         4.2                   28.9          True
   PPE佩戴不规范    182         4.1                   33.0          True
      防护罩缺失    181         4.1                   37.1          True
       泄漏风险    166         3.7                   40.8          True
       通道障碍    158         3.5                   44.3          True
       接地不良    154         3.5                   47.8          True
       设备故障    152         3.4                   51.2          True
       线路老化    145         3.3                   54.5          True
       标识缺失    136         3.1                   57.6          True
       照

In [22]:
# 帕累托图可视化
visualizer = HazardVisualizer()

# 获取帕累托数据
pareto_data = analyzer.get_pareto_analysis('hazard_type')

# 创建帕累托图
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 柱状图 - 数量
colors = ['#e74c3c' if vital else '#3498db' for vital in pareto_data['is_vital_few']]

fig.add_trace(
    go.Bar(
        x=pareto_data['hazard_type'],
        y=pareto_data['count'],
        name='隐患数量',
        marker_color=colors,
        text=pareto_data['count'],
        textposition='outside'
    ),
    secondary_y=False
)

# 折线图 - 累计百分比
fig.add_trace(
    go.Scatter(
        x=pareto_data['hazard_type'],
        y=pareto_data['cumulative_percentage'],
        name='累计百分比',
        mode='lines+markers',
        line=dict(color='#2c3e50', width=2),
        marker=dict(size=6)
    ),
    secondary_y=True
)

# 80%参考线
fig.add_hline(y=80, line_dash='dash', line_color='#e74c3c', 
              annotation_text='80%', secondary_y=True)

fig.update_layout(
    title='<b>隐患类型帕累托分析</b>',
    xaxis_title='隐患类型',
    height=500,
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    xaxis_tickangle=-45
)

fig.update_yaxes(title_text='隐患数量', secondary_y=False)
fig.update_yaxes(title_text='累计百分比 (%)', secondary_y=True, range=[0, 105])

fig.show()


## 4. 区域风险分析

分析各厂区的风险分布情况，识别高风险区域。


In [23]:
# 区域风险汇总（近30天）
area_summary = analyzer.get_area_summary(days=30)

print("各区域风险排名 (近30天)")
print("=" * 80)
display(area_summary.style.background_gradient(subset=['total_risk'], cmap='YlOrRd'))


各区域风险排名 (近30天)


,area_name,total,avg_risk,total_risk,rectified,rectify_rate,high_risk_count
6,硫酸车间,35,3.310000,116,26,74.300000,1
7,萃取车间,31,3.450000,107,23,74.200000,0
5,电解车间,31,3.190000,99,22,71.000000,2
8,镍钴车间,29,3.140000,91,16,55.200000,2
3,废水处理,23,3.090000,71,16,69.600000,0
2,原料仓库,18,3.720000,67,12,66.700000,1
1,公用工程,17,2.940000,50,9,52.900000,0
4,成品仓库,14,3.070000,43,11,78.600000,0
0,中控室,14,3.000000,42,10,71.400000,0


In [24]:
# 区域风险柱状图
fig = px.bar(
    area_summary.sort_values('total_risk', ascending=True),
    x='total_risk',
    y='area_name',
    orientation='h',
    title='<b>各区域风险分数排名 (近30天)</b>',
    labels={'total_risk': '累计风险分', 'area_name': '区域'},
    color='total_risk',
    color_continuous_scale='YlOrRd',
    text='total_risk'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
fig.update_layout(height=400, showlegend=False)
fig.show()


## 5. 隐患类型分布 (Treemap)

使用矩形树图展示隐患大类和具体类型的层级分布。


In [25]:
# 隐患类型分布 Treemap (近90天)
today = datetime.now()
df_recent = df[df['date'] >= today - timedelta(days=90)]

# 按类别和类型聚合
treemap_data = df_recent.groupby(['hazard_category', 'hazard_type']).agg({
    'hazard_id': 'count',
    'risk_score': 'mean'
}).reset_index()
treemap_data.columns = ['category', 'type', 'count', 'avg_risk']

fig = px.treemap(
    treemap_data,
    path=['category', 'type'],
    values='count',
    color='avg_risk',
    color_continuous_scale='RdYlGn_r',
    title='<b>隐患类型分布 (近90天)</b>',
    hover_data={'avg_risk': ':.2f'}
)

fig.update_traces(
    textinfo='label+value',
    hovertemplate='<b>%{label}</b><br>数量: %{value}<br>平均风险分: %{customdata[0]:.2f}<extra></extra>'
)

fig.update_layout(height=500)
fig.show()


## 6. 整改趋势分析

展示各季度的隐患数量和整改率趋势，对比85%目标线。


In [26]:
# 季度趋势图
quarterly = analyzer.get_quarterly_trend()

fig = make_subplots(specs=[[{"secondary_y": True}]])

# 柱状图 - 隐患数量
fig.add_trace(
    go.Bar(
        x=quarterly['quarter'],
        y=quarterly['total'],
        name='隐患数量',
        marker_color='#3498db',
        text=quarterly['total'],
        textposition='outside'
    ),
    secondary_y=False
)

# 折线图 - 整改率
fig.add_trace(
    go.Scatter(
        x=quarterly['quarter'],
        y=quarterly['rectify_rate'],
        name='整改率',
        mode='lines+markers+text',
        line=dict(color='#2ecc71', width=3),
        marker=dict(size=10),
        text=quarterly['rectify_rate'].apply(lambda x: f'{x}%'),
        textposition='top center'
    ),
    secondary_y=True
)

# 85%目标线
fig.add_hline(y=85, line_dash='dash', line_color='#e74c3c',
              annotation_text='目标 85%', secondary_y=True)

fig.update_layout(
    title='<b>季度隐患整改趋势</b>',
    xaxis_title='季度',
    height=450,
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.update_yaxes(title_text='隐患数量', secondary_y=False)
fig.update_yaxes(title_text='整改率 (%)', secondary_y=True, range=[0, 105])

fig.show()


## 7. 3D曲面图分析

展示区域-时间-风险的三维关系，直观展示风险热点。


In [27]:
# 3D曲面图 - 区域 × 时间 × 风险
surface_data = analyzer.get_3d_surface_data()

x_vals = list(range(len(surface_data['x'])))
y_vals = list(range(len(surface_data['y'])))
z_data = np.array(surface_data['z'])

fig = go.Figure(data=[
    go.Surface(
        x=x_vals,
        y=y_vals,
        z=z_data,
        colorscale='RdYlGn_r',
        colorbar=dict(title='风险分'),
        hovertemplate='时间: %{x}<br>区域: %{y}<br>风险分: %{z:.0f}<extra></extra>'
    )
])

fig.update_layout(
    title='<b>3D风险曲面图 (区域 × 时间)</b>',
    scene=dict(
        xaxis=dict(
            title='时间',
            ticktext=surface_data['x'][::3],  # 每3个月显示一个标签
            tickvals=list(range(0, len(surface_data['x']), 3))
        ),
        yaxis=dict(
            title='区域',
            ticktext=surface_data['y'],
            tickvals=y_vals
        ),
        zaxis=dict(title='风险分'),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    height=600
)

fig.show()


## 8. 预警与监控

### 8.1 近7天中高风险预警


In [28]:
# 近7天中高风险预警
recent_alerts = analyzer.get_recent_alerts(days=7, top_n=15)

print(f"近7天发现的中高风险隐患 ({len(recent_alerts)} 条)")
print("=" * 100)

# 格式化显示
if len(recent_alerts) > 0:
    display_df = recent_alerts.copy()
    display_df['date'] = display_df['date'].dt.strftime('%Y-%m-%d')
    display_df['risk_level'] = display_df['risk_level'].map({'High': '[高]', 'Medium': '[中]'})
    display_df.columns = ['日期', '区域', '大类', '类型', '描述', '风险等级', '风险分', '状态']
    display(display_df)
else:
    print("近7天无中高风险隐患")


近7天发现的中高风险隐患 (15 条)


,日期,区域,大类,类型,描述,风险等级,风险分,状态
4451,2026-01-02,原料仓库,作业安全,高处作业违规,原料仓库高处作业未设置警戒,[高],10,整改中
4425,2025-12-29,电解车间,电气安全,临时用电,电解车间发现临时用电未办理审批手续,[高],10,已整改
4430,2025-12-30,镍钴车间,行走安全,台阶损坏,镍钴车间楼梯台阶破损,[高],9,已整改
4433,2025-12-30,硫酸车间,行走安全,通道障碍,硫酸车间安全通道宽度不足,[高],9,已整改
4413,2025-12-27,成品仓库,行走安全,通道障碍,成品仓库通道被物料堆占,[中],7,已整改
4452,2026-01-02,硫酸车间,电气安全,配电箱隐患,硫酸车间配电箱警示标识缺失,[中],6,整改中
4457,2026-01-02,镍钴车间,个人防护,PPE佩戴不规范,镍钴车间作业人员未佩戴安全帽,[中],6,整改中
4458,2026-01-02,中控室,化学品安全,混存问题,中控室酸碱类化学品混存,[中],6,整改中
4423,2025-12-29,镍钴车间,电气安全,临时用电,镍钴车间临时配电箱未接地,[中],6,已整改
4424,2025-12-29,硫酸车间,消防安全,应急照明故障,硫酸车间疏散指示标志损坏,[中],6,已整改


### 8.2 滞留未整改隐患（超7天）


In [29]:
# 超7天未整改的隐患
overdue = analyzer.get_overdue_hazards(min_days=7, top_n=15)

print(f"超7天未整改隐患 ({len(overdue)} 条显示)")
print("=" * 100)

if len(overdue) > 0:
    display_df = overdue.copy()
    display_df['date'] = display_df['date'].dt.strftime('%Y-%m-%d')
    display_df['risk_level'] = display_df['risk_level'].map({
        'High': '[高]', 'Medium': '[中]', 'Low': '[低]'
    })
    display_df.columns = ['编号', '发现日期', '区域', '类型', '风险等级', '风险分', '滞留天数', '责任人', '状态']
    display(display_df)
else:
    print("无超期未整改隐患")


超7天未整改隐患 (15 条显示)


,编号,发现日期,区域,类型,风险等级,风险分,滞留天数,责任人,状态
4410,HZ-2025-R02479,2025-12-26,镍钴车间,漏电保护失效,[中],4,7,施杰,整改中
10,HZ-2024-00011,2024-01-02,成品仓库,设备故障,[低],3,731,喻娟,逾期
86,HZ-2024-00083,2024-01-16,公用工程,PPE佩戴不规范,[低],2,717,刘志强,逾期
298,HZ-2024-00301,2024-02-22,镍钴车间,线路老化,[低],1,680,梁海燕,逾期
319,HZ-2024-00317,2024-02-26,废水处理,动火作业隐患,[低],2,676,何军,逾期
590,HZ-2024-00594,2024-04-18,硫酸车间,混存问题,[低],1,624,王凯,逾期
591,HZ-2024-00589,2024-04-18,废水处理,配电箱隐患,[低],1,624,储云,逾期
604,HZ-2024-00608,2024-04-22,公用工程,锅炉隐患,[低],3,620,张倩,逾期
681,HZ-2024-00682,2024-05-07,公用工程,紧急停止失效,[低],3,605,章楠,逾期
882,HZ-2024-00880,2024-06-12,原料仓库,护栏缺失,[低],3,569,王静,逾期


---

## 9. 总结

### 系统功能概述

| 模块 | 功能 | 输出 |
|------|------|------|
| 数据生成 | 模拟隐患记录 | hazards.csv |
| 风险分析 | 帕累托、趋势、区域汇总 | 统计数据 |
| 可视化 | 热力图、Treemap、3D曲面图 | 交互式图表 |
| 仪表板 | HTML报告生成 | risk_dashboard.html |

### 关键发现

运行以下代码查看本次分析的关键发现：


In [30]:
# 关键发现汇总
print("=" * 60)
print("关键发现汇总")
print("=" * 60)

# 1. 高风险区域
top_area = area_summary.iloc[0]
print(f"\n[1] 最高风险区域: {top_area['area_name']}")
print(f"    风险总分: {top_area['total_risk']:.0f}")
print(f"    高风险隐患: {top_area['high_risk_count']} 条")

# 2. 关键少数隐患类型
print(f"\n[2] 关键少数 (占80%问题的隐患类型):")
for i, row in vital_few.head(3).iterrows():
    print(f"    - {row['hazard_type']}: {row['count']}条 ({row['percentage']}%)")

# 3. 整改率情况
print(f"\n[3] 整改情况:")
print(f"    总整改率: {stats['rectify_rate']}%")
print(f"    平均整改天数: {stats['avg_rectify_days']} 天")
print(f"    目标整改率: 85%")
if stats['rectify_rate'] >= 85:
    print(f"    状态: 达标")
else:
    print(f"    状态: 未达标 (差距: {85 - stats['rectify_rate']:.1f}%)")

# 4. 待处理事项
overdue_count = len(analyzer.get_overdue_hazards(7))
recent_high_count = len(analyzer.get_recent_alerts(7))
print(f"\n[4] 待处理事项:")
print(f"    近7天中高风险: {recent_high_count} 条")
print(f"    超7天未整改: {overdue_count} 条")

print("\n" + "=" * 60)
print("分析完成。运行 main.py 可生成完整HTML仪表板。")
print("=" * 60)


关键发现汇总

[1] 最高风险区域: 硫酸车间
    风险总分: 116
    高风险隐患: 1 条

[2] 关键少数 (占80%问题的隐患类型):
    - 临时用电: 340条 (7.6%)
    - 灭火器过期: 268条 (6.0%)
    - 配电箱隐患: 253条 (5.7%)

[3] 整改情况:
    总整改率: 97.5%
    平均整改天数: 16.9 天
    目标整改率: 85%
    状态: 达标

[4] 待处理事项:
    近7天中高风险: 15 条
    超7天未整改: 50 条

分析完成。运行 main.py 可生成完整HTML仪表板。
